In [1]:

# Set up notebook
%pprint
import sys
if (osp.join('..', 'py') not in sys.path): sys.path.insert(1, osp.join('..', 'py'))
from FRVRS import (nu, fu, re)
nu.delete_ipynb_checkpoint_folders()

Pretty printing has been turned OFF


In [7]:

# Extract all the needed columns from all the function calls in the function
import inspect
import pyperclip

functions_regex = re.compile(r'\bself\.[a-z_]+\([^\)]+_df[^\)]*\)')
issubset_regex = re.compile(r'\.issubset\(')
needed_columns_set = set()
fn_obj = fu.create_triage_error_rates_dataframe
source_code = inspect.getsource(fn_obj)
parts_list = functions_regex.findall(source_code)
statements_list = [f'These functions still need code to ensure all needed columns can be checked in {fn_obj.__name__}:']
for fn_str in parts_list:
    fn_obj = eval(fn_str.split('(')[0].replace('self.', 'fu.'))
    source_code = inspect.getsource(fn_obj)
    parts_list = re.split('(needed_columns = |all_columns = )', source_code, 0)
    if (issubset_regex.search(source_code) is None) or (len(parts_list) < 2):
        statements_list.append('    ' + fn_obj.__name__)
    else:
        needed_columns_str = parts_list[2].strip()
        try:
            needed_columns_set = needed_columns_set.union(eval(re.sub(r'\bself\.', 'fu.', needed_columns_str)))
        except:
            print(needed_columns_str)
if len(statements_list) > 1: print('\n'.join(statements_list))
elif len(needed_columns_set):
    print_str = f"""
        
        # Ensure all needed columns are present in injury_df
        needed_columns = set({list(needed_columns_set)})
        all_columns = set(injury_df.columns)
        assert needed_columns.issubset(all_columns), f"You're missing {{needed_columns.difference(all_columns)}} from injury_df"
"""
    pyperclip.copy(print_str)
    print(print_str)


        
        # Ensure all needed columns are present in injury_df
        needed_columns = set(['patient_count', 'error_type'])
        all_columns = set(injury_df.columns)
        assert needed_columns.issubset(all_columns), f"You're missing {needed_columns.difference(all_columns)} from injury_df"

